In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#样本大，特征此例子为1个，但是也适用于多特征
m = 100000
np.random.seed(666)
x = np.random.random(size = m)
X = x.reshape(-1, 1) #转换成一列，100个样本（100行），1个特征（1列）
y = 4. * x + 3. + np.random.normal(size = m)

In [34]:
def J(theta, X_b, y):  # 损失函数J，其中X_b是已经加了第一列是1
    try:
        y_hat = X_b.dot(theta)
        return np.sum((y - y_hat) ** 2) / len(X_b)
    except:
        return float('inf')

# def dJ(theta, X_b, y): #非向量化
#     res = np.empty(len(theta))
#     res[0] = np.sum(X_b.dot(theta) - y)
#     for i in range(1, len(theta)):
#         res[i] = (X_b.dot(theta) - y).dot(X_b[:, i])  # 将(X_b.dot(theta) - y)看做行向量，而不是列向量
#     return res / len(X_b) * 2

def dJ(theta, X_b, y): #向量化
    return X_b.T.dot(X_b.dot(theta) - y) / len(X_b) * 2.

def gradient_descent(X_b, y, initial_theta, eta, n_iters = 1e4, epsilon = 1e-8):
    theta = initial_theta
    i_iter = 0
    while i_iter <= n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta + (-1) * eta * gradient
        if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        i_iter += 1

    return theta

In [35]:
%%time
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
theta = gradient_descent(X_b, y, initial_theta, eta)

CPU times: user 22.9 s, sys: 78.1 ms, total: 23 s
Wall time: 3.84 s


### Stochastic Gradient Descent

In [11]:
def dJ_sgd(theta, X_b_i, y_i): #传入的不是X_b整个矩阵而是X_b的第i个样本，y也变成y_i
    return X_b_i.T.dot(X_b_i.dot(theta) - y_i) / len(X_b_i) * 2.

In [28]:
def sgd(X_b, y, initial_theta, n_iters): #不需要传入学习率eta，是里面自己计算的
    t0 = 5
    t1 = 50
    
    def learning_rate(t):
        return t0 / (t + t1)
    
    theta = initial_theta
    
    """为什么不需要下面这一段：这一段的break条件：1. i_iters 超过 n_iters 2. abs()差值足够小
    while i_iter <= n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta + (-1) * eta * gradient
        if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon): #因为采用sgd，下降方向是随机的，即便abs(xx-yy)的值很小，也不代表到了最低值
            break
        i_iter += 1
    """
    #现在break条件： i_iters 超过 n_iters （所以用for loop）
    for cur_iter in range(n_iters):
        rand_i = np.random.randint(len(X_b))
        gradient = dJ_sgd(theta, X_b[rand_i], y[rand_i])
        theta = theta + (-1) * learning_rate(cur_iter) * gradient
        
    return theta
    

In [29]:
%%time
X_b = np.hstack([np.ones((len(X),1)), X])
initial_theta = np.zeros(X_b.shape[1])
theta = sgd(X_b, y, initial_theta, n_iters = (int)(len(X_b) / 3)) #不使用全部m个样本，而是使用1/3

CPU times: user 231 ms, sys: 2.99 ms, total: 234 ms
Wall time: 234 ms


In [31]:
X_b = np.hstack([np.ones((len(X),1)), X])
initial_theta = np.zeros(X_b.shape[1])
theta = sgd(X_b, y, initial_theta, n_iters = (int)(len(X_b) / 3)) #不使用全部m个样本，而是使用1/3
theta #[3.11406959, 3.77094008]接近真值[3,4]

array([3.10829085, 3.8215458 ])